In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from utils.PredictSalesUtils import getTestEnriched
from utils.PredictSalesUtils import cleanEnsembleDataset, cleanStackingDataset
from utils.PredictSalesUtils import generateFeaturesForEvaluation

In [ ]:
import math

In [ ]:
import sys
sys.path.append('../dsbase/src/main')

In [ ]:
from dsbase.models.regression.LightGradientBoostingRegressionDSBase import LightGradientBoostingRegressionDSBaseModel
from dsbase.models.regression.DNNRegressionKerasDSBase import DNNRegressionKerasDSBaseModel

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

# Test Data Loading 

In [ ]:
test_df_enriched = getTestEnriched('datasets/test.csv','datasets/items.csv')

In [ ]:
test_df_enriched.head()

# Sales Direct Load

In [ ]:
sales_df = pd.read_csv('datasets/sales_train_enriched.csv')

In [ ]:
sales_df.drop(labels=['Unnamed: 0'], inplace=True, axis=1)

In [ ]:
sales_df.head()

# Evaluation Processing 

## Ensemble Process

In [ ]:
slots = sales_df['date_block_num'].nunique()
slot_seq = np.arange(1,slots)

In [ ]:
windows = [3,6,12,18,25,32]

In [ ]:
def processFold(i):
    months = slot_seq[-windows[i-1]:]
    fold_df = generateFeaturesForEvaluation(sales_df, months, test_df_enriched)
    
    #sns.heatmap(~fold_df.isna())
    fold_df_cleaned = cleanEnsembleDataset(fold_df, imputeTarget=False)
    fold_df_cleaned.set_index(keys=['ID_pair'], inplace=True)
    
    # Prediction in model fold[index]
    X = fold_df_cleaned.values
    model = LightGradientBoostingRegressionDSBaseModel('model' + str(i) + 'A0')
    model.load()
    y_pred = model.predict(X)
    
    column_name = 'target_prediction'
    fold_df_cleaned_completed = pd.concat(objs=[fold_df_cleaned.reset_index(),pd.DataFrame(y_pred,columns=[column_name])], axis=1)
    fold_df_cleaned_completed[['ID_pair',column_name]]        
    return fold_df_cleaned_completed[['ID_pair',column_name]]

### Fold 1

In [ ]:
df1_target = processFold(1)

### Fold 2

In [ ]:
df2_target = processFold(2)

### Fold 3

In [ ]:
df3_target = processFold(3)

### Fold 4

In [ ]:
df4_target = processFold(4)

### Fold 5

In [ ]:
df5_target = processFold(5)

### Fold 6

In [ ]:
df6_target = processFold(6)

Now, let's join the result for the stacking stage: 

In [ ]:
df_for_stacking = df1_target.merge(
    right=df2_target, on=['ID_pair'],how='outer', suffixes=('_1','_2')
).merge(
    right=df3_target, on=['ID_pair'],how='outer'
).merge(
    right=df4_target, on=['ID_pair'],how='outer', suffixes=('_3','_4')
).merge(
    right=df5_target, on=['ID_pair'],how='outer'
).merge(
    right=df6_target, on=['ID_pair'],how='outer', suffixes=('_5','_6')
)

In [ ]:
df_for_stacking_cleaned = cleanStackingDataset(df_for_stacking,['target_prediction_1','target_prediction_2','target_prediction_3','target_prediction_4','target_prediction_5','target_prediction_6'])

In [ ]:
df_for_stacking_cleaned.to_csv('datasets/eval_dataset_for_stacking_cleaned.csv')

## Stacking Process 

In [ ]:
df_for_stacking_cleaned = pd.read_csv('datasets/eval_dataset_for_stacking_cleaned.csv')

In [ ]:
df_for_stacking_cleaned.drop(labels=['Unnamed: 0'], inplace=True, axis=1)

In [ ]:
df_for_stacking_cleaned.set_index(keys=['ID_pair'], inplace=True)

In [ ]:
X = df_for_stacking_cleaned.values

In [ ]:
scalerX = joblib.load('model_persistance/dnn_scalerX.sav')
scalery = joblib.load('model_persistance/dnn_scalery.sav')

In [ ]:
model = DNNRegressionKerasDSBaseModel('DNN3')
model.load()

In [ ]:
y_s_pred = model.predict(scalerX.transform(X))

In [ ]:
y_pred = scalery.inverse_transform(y_s_pred)

In [ ]:
predictions = pd.concat(objs=[df_for_stacking_cleaned.reset_index(),pd.DataFrame(y_pred, columns=['predictions'])], axis=1)

In [ ]:
test_df_enriched_predictions = test_df_enriched.merge(right=predictions, on='ID_pair', how='left')

In [ ]:
pre_submission = test_df_enriched_predictions[['ID','predictions']]

In [ ]:
def finalFitting(x):
    if (math.isnan(x)):
        return 0
    elif (x < 0):
        return 0
    else:
        return x

In [ ]:
pre_submission['item_cnt_month'] = pre_submission['predictions'].apply(finalFitting)

In [ ]:
submission = pre_submission[['ID','item_cnt_month']]

In [ ]:
submission.to_csv('datasets/submission.csv', index=False)

# End of Case! 